# Development notes for pyxelate

## Display ratio calculation

Round to quarters for ratios below 3, to halves for ratios below 5, and to integer numbers for all greater ratios.

In [1]:
import math
from fractions import Fraction

uproundable = Fraction(17, 10)
exact = Fraction(15, 10)

uprounded_ratio = Fraction(math.ceil(uproundable * 4), 4)
exact_ratio = Fraction(math.ceil(exact * 4), 4)

print('Uprounded %s: %s' % (uproundable, uprounded_ratio))
print('Exact %s: %s' % (exact, exact_ratio))

Uprounded 17/10: 7/4
Exact 3/2: 3/2


In [3]:
def display_ratio(image_size, canvas_size):
    """Display ratio calculated per dimension"""
    if image_size < canvas_size:
        return 1
    #
    raw_ratio = Fraction(image_size, canvas_size)
    if raw_ratio <= 3:
        return Fraction(math.ceil(raw_ratio * 4), 4)
    #
    if raw_ratio <= 5:
        return Fraction(math.ceil(raw_ratio * 2), 2)
    #
    return math.ceil(raw_ratio)

In [4]:
print(display_ratio(17, 10))
print(display_ratio(17, 5))
print(display_ratio(17, 3))

7/4
7/2
6
